# Aplicaciones de Transformers en Procesamiento de Lenguaje Natural

Este cuaderno explora distintas aplicaciones de los modelos Transformers en tareas de procesamiento de lenguaje natural (PLN), utilizando ejemplos prácticos y explicaciones teóricas.

---

## ¿Qué es la arquitectura Transformer?

La **arquitectura Transformer** es un tipo de red neuronal propuesta por Vaswani et al. en 2017. Su principal innovación es el mecanismo de *atención*, que permite al modelo enfocarse en diferentes partes de la entrada para procesar información contextual de manera eficiente. Los Transformers reemplazaron a modelos anteriores como LSTM y GRU en muchas tareas de PLN debido a su capacidad para manejar secuencias largas y aprender dependencias complejas.

---

## ¿Qué es la librería Transformers de Hugging Face?

La **librería Transformers de Hugging Face** es una biblioteca de Python que proporciona implementaciones preentrenadas de modelos basados en la arquitectura Transformer (como BERT, GPT, T5, etc.) para tareas de PLN. Permite cargar modelos fácilmente y utilizarlos en tareas como clasificación, traducción, resumen, etc., sin necesidad de entrenarlos desde cero. Podes explorar los modelos disponibles en la [página de modelos de Hugging Face](https://huggingface.co/models).

---

## ¿Qué es un pipeline en Hugging Face?

Un **pipeline** en Hugging Face es una interfaz de alto nivel que simplifica el uso de modelos preentrenados para tareas específicas. Permite ejecutar tareas como clasificación de texto, reconocimiento de entidades, traducción, etc., con solo unas pocas líneas de código, ocultando detalles complejos de preprocesamiento y postprocesamiento.


## 1. Definir el texto de ejemplo

A continuación, se presenta un texto de ejemplo que será utilizado en las distintas aplicaciones de modelos Transformers. Este texto simula una queja de un cliente sobre un pedido equivocado, lo que nos permitirá explorar tareas como clasificación, reconocimiento de entidades, respuesta a preguntas, resumen, traducción y generación de texto.

In [1]:
texto = """Querido MercadoLibre, la semana pasada pedí una figura de acción de Optimus Prime desde su tienda online.
Para mi sorpresa, cuando abrí el paquete, descubrí horrorizado que me habían enviado una figura de Megatron.
Como fan de los Autobots, espero que entiendan mi decepción. Solicito un cambio urgente del producto."""

## 2. Clasificación de texto con Transformers

La **clasificación de texto** es una tarea fundamental en PLN que consiste en asignar una o varias etiquetas a un texto, como por ejemplo identificar el sentimiento (positivo, negativo, neutro) o la intención del mensaje. Los modelos Transformers, gracias a su capacidad para comprender el contexto, demostraron un rendimiento sobresaliente en esta tarea.

Utilizaremos el pipeline `text-classification` de la librería 🤗 Transformers para analizar el texto de ejemplo y visualizar el resultado de la clasificación.

In [2]:
from transformers import pipeline
# Modelo de clasificación de texto (sentimiento) en español
# Puedes buscar otros modelos en https://huggingface.co/models
classifier = pipeline("text-classification", model="pysentimiento/robertuito-sentiment-analysis")
import pandas as pd
outputs = classifier(texto)
pd.DataFrame(outputs)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/925 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/435M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/384 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

Device set to use cpu


,label,score
0,NEG,0.913063


## 3. Reconocimiento de entidades nombradas (NER)

El **Reconocimiento de Entidades Nombradas** (NER, por sus siglas en inglés) es la tarea de identificar y clasificar automáticamente nombres propios, lugares, organizaciones y otras entidades relevantes dentro de un texto. Los Transformers permiten realizar NER de manera eficiente y precisa.

A continuación, aplicamos el pipeline `ner` al texto de ejemplo para extraer las entidades nombradas presentes.

In [3]:
# Modelo de reconocimiento de entidades nombradas (NER) en español
# Puedes buscar otros modelos en https://huggingface.co/models
ner_tagger = pipeline("ner", model="mrm8488/bert-spanish-cased-finetuned-ner", aggregation_strategy="simple")
outputs = ner_tagger(texto)
pd.DataFrame(outputs)

config.json:   0%|          | 0.00/829 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

Some weights of the model checkpoint at mrm8488/bert-spanish-cased-finetuned-ner were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/136 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/439M [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cpu
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


,entity_group,score,word,start,end
0,ORG,0.756585,MercadoLibre,8,20
1,MISC,0.829108,Optimus Prime,68,81
2,MISC,0.763651,Megatron,205,213
3,MISC,0.954693,Autobot,231,238


## 4. Respuesta a preguntas basada en contexto

La tarea de **respuesta a preguntas** (Question Answering) consiste en responder preguntas específicas utilizando un contexto dado. Los modelos Transformers pueden comprender el texto y extraer la información relevante para responder de manera precisa.

En el siguiente ejemplo, preguntamos al modelo qué desea el cliente, utilizando el pipeline `question-answering`.

In [4]:
# Modelo de respuesta a preguntas en español
# Puedes buscar otros modelos en https://huggingface.co/models
reader = pipeline("question-answering", model="PlanTL-GOB-ES/roberta-large-bne-sqac")
question = "¿Qué quiere el cliente?"
outputs = reader(question=question, context=texto)
pd.DataFrame([outputs])

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

Device set to use cpu


,score,start,end,answer
0,0.999948,285,315,un cambio urgente del producto


## 5. Resumen automático de texto

El **resumen automático** permite condensar la información principal de un texto extenso en una versión más corta, manteniendo el significado esencial. Los Transformers han revolucionado esta tarea gracias a su capacidad de comprensión contextual.

A continuación, utilizamos el pipeline `summarization` para obtener un resumen del texto de ejemplo.

In [5]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
# Puedes buscar otros modelos en https://huggingface.co/models
model_name = "csebuetnlp/mT5_multilingual_XLSum"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
resumidor = pipeline("summarization", model=model, tokenizer=tokenizer)
resumen = resumidor(texto, max_length=80, min_length=20, do_sample=False)
print(resumen[0]['summary_text'])

tokenizer_config.json:   0%|          | 0.00/375 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/730 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/2.33G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.33G [00:00<?, ?B/s]

Device set to use cpu


Hace unos meses, un paquete de acciones de la tienda online Amazon me envió una figura de Megatron.


## 6. Traducción automática del español a inglés

La **traducción automática** es una de las aplicaciones más destacadas de los Transformers, permitiendo traducir textos entre diferentes idiomas con alta calidad. Utilizaremos el pipeline `translation_es_to_en` para traducir el texto de ejemplo del español al inglés.

In [6]:
# Modelo de traducción de español a inglés
# Puedes buscar otros modelos en https://huggingface.co/models
translator = pipeline("translation_es_to_en", model="Helsinki-NLP/opus-mt-es-en")
outputs = translator(texto)
print(outputs[0]['translation_text'])

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/312M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/312M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/826k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:177: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
Device set to use cpu


Dear MarketFree, last week I asked for an action figure of Optimus Prime from his online store. To my surprise, when I opened the package, I discovered horrified that I had been sent a figure of Megatron. As a fan of Autobots, I hope you understand my disappointment. I request an urgent change of product.


## 7. Generación automática de texto (respuesta de servicio al cliente)

La **generación automática de texto** permite crear o continuar textos de manera coherente a partir de un prompt inicial. Esta capacidad es útil, por ejemplo, para redactar respuestas automáticas en atención al cliente.

En el siguiente ejemplo, generamos una posible respuesta de servicio al cliente utilizando el pipeline `text-generation`.

In [7]:
# Modelo de generación de texto en español
# Puedes buscar otros modelos en https://huggingface.co/models
generator = pipeline("text-generation", model="datificate/gpt2-small-spanish")
respuesta_inicial = "Estimado cliente, lamentamos mucho lo ocurrido con su pedido. "
prompt = texto + "\n\nRespuesta del servicio al cliente:\n" + respuesta_inicial
outputs = generator(
    prompt,
    max_new_tokens=150,
    do_sample=True,
    temperature=0.7,
    top_k=50,
    top_p=0.9,
    repetition_penalty=1.3,
    eos_token_id=50256
)
print(outputs[0]['generated_text'])

config.json:   0%|          | 0.00/817 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/510M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/620 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/510M [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

Device set to use cpu


Querido MercadoLibre, la semana pasada pedí una figura de acción de Optimus Prime desde su tienda online.
Para mi sorpresa, cuando abrí el paquete, descubrí horrorizado que me habían enviado una figura de Megatron.
Como fan de los Autobots, espero que entiendan mi decepción. Solicito un cambio urgente del producto.

Respuesta del servicio al cliente:
Estimado cliente, lamentamos mucho lo ocurrido con su pedido.  Salud a la venta y se da cuenta de las nuevas necesidades para este producto final...
Este es uno de nuestros momentos más emocionantes e interesantes dentro de la franquicia Transformers como siempre ha sido".
"Con esta entrega nos enteramos sobre lo mejor de la segunda temporada."
La primera temporada cuenta junto a "En busca de Cybertron", "Transformer" (2013), "The King of the Banshees" (2014), "Dynamite" (2015-2017).
También cuenta también algunos episodios dedicados exclusivamente a Transforms y a otros personajes como "", "" (2018 ), "Crossroads" (2019 ) o "Tak Tokaren" 